# 時期ごとの極性・共起変遷を追う

## 必要なもの
* ID, POPLATION_ID, PERIOD_IDのセット
* local_stopwordsのセット
* 分析したキーワードをKeywords/Timelapse/ID-from-POPULATION_ID.txtにスペース区切りで保存
* get_time_index()の書き換え

In [ ]:
ID = ""

POPULATION_ID = ""

PERIOD_ID = ""

#分析対象の単語ファイル
keywords_file = "Keywords/Timelapse/%s-from-%s.txt"%(ID, POPULATION_ID)

#ストップワード抜きの形態素解析・極性分析完了したデータ
without_stopwords_text_file = "Progresses/NonStopword/%s-from-%s.txt"%(ID, POPULATION_ID)

##TFIDFモデル
tfidf_model_file = "Models/Tfidf/%s-from-%s.model"%(ID, POPULATION_ID)

#保存するファイル
sentiment_timelapse_file = "Sentiment/Timelapse/%s-from-%s-by-%s.txt"%(ID, POPULATION_ID, PERIOD_ID)

#内部的に品詞を区別する区切り文字
TOKEN_DIVIDER = "<334>"

In [ ]:
#共起キーワードのストップワード（ここだけ）
local_stopwords = ["br", "ミミッキュ", "ミミ"]

## 

## 与えられた時期にインデックスを割り振る
分けたい時期にあわせて書き換える

In [ ]:
from datetime import datetime

In [ ]:
#これを書き換える
#万が一 -1 が出てしまうとエラーが出てしまうので注意（上限はない）
def get_time_index(time: datetime):
    #2000年から一年ごとに割り振る
    if time == None:
        return None

    years_passed = time.year - 2000
    
    if years_passed >= 0:
        return years_passed
    else:
        return 0

#保持しているデータの形式からdatetime型に変換
def convert_time(text: str):
    if (text == None) | (text == "None"):
        return None
    else:
        return datetime.strptime(text, "%Y/%m/%d|%H/%M/%S")

#最大インデックスを算出
MAX_TIME_INDEX = get_time_index(datetime.now())

## データ読み込み

In [ ]:
import pandas as pd
import numpy as np

### 処理済みデータ

In [ ]:
df_main = pd.read_table(without_stopwords_text_file)

df_main.head(2)

### 本ノート用キーワード

In [ ]:
with open(keywords_file, "r", encoding="utf-8") as f:
    keywords = f.read().split()

keywords

### 母集団からのTFIDFモデルをサルベージ

In [ ]:
import pickle
from copy import deepcopy

def tokenize(text):
    return text.split(TOKEN_DIVIDER)

with open(tfidf_model_file, "rb") as f:
    tfidf_model = pickle.load(f)

#対象（名詞・形容詞）を見せる用
tfidf_model_showing = deepcopy(tfidf_model)
tfidf_model_showing.ngram = (1,1)
tfidf_model_showing.tokenizer = tokenize

## 分析

### 各テキストに時期インデックスを割り振る

In [ ]:
indices = []

for _, row in df_main.iterrows():
    indices.append(get_time_index(convert_time(row["date"])))

df_main["time_index"] = indices

df_main.head(3)

### キーワードごとに分析

In [ ]:
#キーワードごとに
for keyword in keywords:
    scores = []
    coocs = []
    counts = []

    #時期インデックスごとに
    for time_index in range(MAX_TIME_INDEX):
        #極性スコア計算
        score = 0
        cnt = 0
        texts = []
        for _, row in df_main[df_main["time_index"] == time_index].iterrows():
            if keyword in row["disassembled_all"].split(TOKEN_DIVIDER):
                cnt += 1
                if row["Sentiment"] == "pos":
                    score += 1
                elif row["Sentiment"] == "neg":
                    score -= 1
                texts.append(row["disassembled_target"])
        if cnt != 0:
            score /= cnt
        else:
            score = 0
        
        ##共起
        #TFIDF計算
        if cnt != 0:
            tfidf_spycy = tfidf_model_showing.transform(texts)
            tfidf_df = pd.DataFrame(tfidf_spycy.toarray(), columns=tfidf_model_showing.get_feature_names_out())
            tfidf_mean = tfidf_df.mean(axis=0)

            #単語名と結び付け
            tfidf_mean = tfidf_mean.sort_values(ascending=False)

            #ストップワードの削除
            for word in local_stopwords:
                try:
                    tfidf_mean = tfidf_mean.drop(word)
                except:
                    pass

            if len(tfidf_mean) > 1:
                cooc = ", ".join(tfidf_mean[1:min(6, len(tfidf_mean))].index)
            else:
                cooc = ""
        else:
            cooc = ""
        
        scores.append(score)
        coocs.append(cooc)
        counts.append(cnt)

    df_keyword = pd.DataFrame(list(zip(counts, scores, coocs)), columns=["Scores", "Counts", "CoOccure"])

    print(keyword)
    print(df_keyword)